In [1]:
!pip install flwr[simulation] --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<

In [37]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, random_split, TensorDataset
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

import flwr as fl
from flwr.common import Metrics, Context

In [3]:
DEVICE = torch.device('cuda')

In [35]:
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"rajaxarcmu","key":"68d40c5e38e1c786ab57736bc5c9b2cb"}')
    
# !chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d 'danofer/compass'
!unzip -qo /kaggle/working/compass.zip -d '/kaggle/content'

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Dataset URL: https://www.kaggle.com/datasets/danofer/compass
License(s): DbCL-1.0
compass.zip: Skipping, found more recently modified local copy (use --force to force download)


In [36]:
!ls /kaggle/content

compas-scores-raw.csv	cox-violent-parsed_filt.csv
cox-violent-parsed.csv	propublicaCompassRecividism_data_fairml.csv


In [28]:
df = pd.read_csv('/kaggle/content/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv')
print(df.shape)
df.head()

(6172, 12)


,Two_yr_Recidivism,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
0,0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,1,4,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1
4,1,14,1,0,0,0,0,0,0,0,0,0


In [93]:
NUM_CLIENTS = 10
# REPRESENTS SILO'D ORGANIZATIONS

In [71]:
trainset, testset = train_test_split(df, test_size=0.2)

part_size = len(trainset) // NUM_CLIENTS
lengths = [part_size] * NUM_CLIENTS
trainset = trainset.sample(sum(lengths))

df_shuffled = trainset.sample(frac=1, random_state=42).reset_index(drop=True)
datasets = np.array_split(df_shuffled, NUM_CLIENTS)

train_loaders = []
val_loaders = []

feature_columns = ['Number_of_Priors', 'score_factor','Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'Misdemeanor']

for ds in datasets:
    train_x = trainset[feature_columns].values
    train_y = trainset['Two_yr_Recidivism'].values
    
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25, shuffle=True, stratify=train_y, random_state=42)
    train_x_tensor = torch.from_numpy(train_x).float()
    train_y_tensor = torch.from_numpy(train_y).float()

    valid_x_tensor = torch.from_numpy(valid_x).float()
    valid_y_tensor = torch.from_numpy(valid_y).float()

    # Create TensorDataset and DataLoader
    train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
    valid_dataset = TensorDataset(valid_x_tensor, valid_y_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

    train_loaders.append(train_loader)
    val_loaders.append(valid_loader)

test_x = testset[feature_columns].values
test_y = testset['Two_yr_Recidivism'].values
test_x_tensor = torch.from_numpy(test_x).float()
test_y_tensor = torch.from_numpy(test_y).float()
test_dataset = TensorDataset(test_x_tensor, test_y_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [89]:
class BaselineNN(nn.Module):
    def __init__(self):
        super(BaselineNN, self).__init__()
        self.fc1 = nn.Linear(5, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

def train(net, trainloader, epochs, verbose=True):
    """
    Train Network on Training Set
    """
    criterion = nn.BCELoss()
    optimizer = optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            labels = labels.view(-1, 1)  # Reshape labels to match outputs
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

def test(net, testloader):
    """
    Test Network on Test Set
    """
    criterion = nn.BCELoss()
    net.eval()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = net(inputs)
            labels = labels.view(-1, 1)  # Reshape labels to match outputs
            loss += criterion(outputs, labels).item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    acc = correct / total
    print(f"Test Loss: {loss:.4f} - Acc: {acc:.4f}")
    return loss, acc

# Centralized Learning

In [97]:
model = BaselineNN()

In [98]:
train_loader = train_loaders[0]
val_loader = val_loaders[0]
model = model.to(DEVICE)
epochs = 10

for epoch in range(epochs):
    train(model, train_loader, 1, verbose=False)
    loss, acc = test(model, val_loader)
    print(f"\nEpoch {epoch+1}/{epochs} - Loss: {loss:.4f} - Acc: {acc:.4f}")

loss, acc = test(model, test_loader)
print(f"Final Performance - Loss: {loss:.4f} - Acc: {acc:.4f}")

Test Loss: 0.6700 - Acc: 0.5606

Epoch 1/10 - Loss: 0.6700 - Acc: 0.5606
Test Loss: 0.6457 - Acc: 0.6636

Epoch 2/10 - Loss: 0.6457 - Acc: 0.6636
Test Loss: 0.6282 - Acc: 0.6766

Epoch 3/10 - Loss: 0.6282 - Acc: 0.6766
Test Loss: 0.6176 - Acc: 0.6766

Epoch 4/10 - Loss: 0.6176 - Acc: 0.6766
Test Loss: 0.6101 - Acc: 0.6857

Epoch 5/10 - Loss: 0.6101 - Acc: 0.6857
Test Loss: 0.6063 - Acc: 0.6909

Epoch 6/10 - Loss: 0.6063 - Acc: 0.6909
Test Loss: 0.6019 - Acc: 0.6883

Epoch 7/10 - Loss: 0.6019 - Acc: 0.6883
Test Loss: 0.6005 - Acc: 0.6902

Epoch 8/10 - Loss: 0.6005 - Acc: 0.6902
Test Loss: 0.5995 - Acc: 0.6889

Epoch 9/10 - Loss: 0.5995 - Acc: 0.6889
Test Loss: 0.5985 - Acc: 0.6876

Epoch 10/10 - Loss: 0.5985 - Acc: 0.6876
Test Loss: 0.6084 - Acc: 0.6810
Final Performance - Loss: 0.6084 - Acc: 0.6810


# Federated Learning with Flower

In [102]:
def get_params(net):
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_params(net, params):
    params_dict = zip(net.state_dict().keys(), params)
    state_dict = OrderedDict({k: torch.Tensor(v) for k,v in params_dict})
    net.load_state_dict(state_dict, strict=True)
    
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
    
    def get_parameters(self, config):
        return get_params(self.net)
    
    def fit(self, parameters, config):
        set_params(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_params(self.net), len(self.trainloader), {}
    
    def evaluate(self, parameters, config):
        set_params(self.net, parameters)
        loss, acc = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {'accuracy': float(acc)}


In [109]:
def client_fn(cid):
    net = BaselineNN().to(DEVICE)
    trainloader = train_loaders[int(cid)]
    valloader = val_loaders[int(cid)]
    return FlowerClient(net, trainloader, valloader)

In [111]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10
)

client_resources = {'num_gpus': 2, 'num_cpus': 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources
)

INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
/opt/conda/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-10-25 13:10:46,833	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 8555921817.0, 'CPU': 4.0, 'accelerator_type:T4': 1.0, 'memory': 17111843636.0, 'GPU': 2.0, 'node:172.19.2.2': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 2, 'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppA

(ClientAppActor pid=4886) Epoch 1/1 - Loss: 0.6965 - Acc: 0.5602
(ClientAppActor pid=4886) Epoch 1/1 - Loss: 0.6946 - Acc: 0.5629


(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=4886) 
(ClientAppActor pid=4886)             This is a deprecated feature. It will be removed
(ClientAppActor pid=4886)             entirely in future versions of Flower.
(ClientAppActor pid=4886)         
(ClientAppActor pid=4886) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=4886) Epoch 1/1 - Loss: 0.6971 - Acc: 0.5605


(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=4886) 
(ClientAppActor pid=4886)             This is a deprecated feature. It will be removed
(ClientAppActor pid=4886)             entirely in future versions of Flower.
(ClientAppActor pid=4886)         
(ClientAppActor pid=4886) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Cont

...

(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=4886) 
(ClientAppActor pid=4886)             This is a deprecated feature. It will be removed
(ClientAppActor pid=4886)             entirely in future versions of Flower.
(ClientAppActor pid=4886)         
(ClientAppActor pid=4886) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Cont

(ClientAppActor pid=4886) Test Loss: 0.6128 - Acc: 0.6798


History (loss, distributed):
	round 1: 0.6728374037894042
	round 2: 0.6453724035332903
	round 3: 0.6279543805106924
	round 4: 0.6190216140320606
	round 5: 0.6127834539095148

(ClientAppActor pid=4886) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=4886) 
(ClientAppActor pid=4886)             This is a deprecated feature. It will be removed
(ClientAppActor pid=4886)             entirely in future versions of Flower.
(ClientAppActor pid=4886)         
(ClientAppActor pid=4886) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=4886) Test Loss: 0.6128 - Acc: 0.6798
(ClientAppActor pid=4886) Test Loss: 0.6128 - Acc: 0.6798
